links:
+ http://naelshiab.com/tutorial-send-email-python/
+ https://docs.python.org/3/library/smtplib.html

** scrape wg-gesucht, **

goal is to be alerted when new listings come online...to be alerted quickly..

+ point is to be alerted when a new listing is posted

requirements:
+ collect links of appropriate properties with following attributes
    - 1 - 2 rooms
    - available at latest 01/07
    - that the rental is long term
    - listings of only the last 1 hour? - sent from 8am to midnight CET?
    - $m^2 $
    - region
    - date and time of listing
    - check that it isn't a swap  (TAUSCH/SWAP)


+ want to run this from the whatbox slot
+ will need to have logging and so on - somehow only new listing should be emailed?
+ want to email the results to myself - especially for certain areas?

** where the information on each listing is **

* ang_spalte_datum row_click
* ang_spalte_miete row_click
* ang_spalte_groesse row_click
* ang_spalte_stadt row_click
* ang_spalte_freiab row_click

** server information **

source ~/virtualenv/bin/activate    ### to activate the virtualenv required for pip and so on

In [ ]:
## TODO:
# function to check if the first listing has been changed
# would be good to send only new listings?
# could store the previous listings in a csv and check the link somehow?
###        dont know if the link could change for the same listing? - but will find out

# implement logging? - what if something fails? - would be good to write what fails
# also perhaps to suspend if the failure of the script occurs?

In [4]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [8]:
def get_search_results(url):
    r = requests.get(url)
    assert r.status_code == 200, 'status code not 200'
    html_doc = r.text
    return BeautifulSoup(html_doc, 'html.parser')


def get_listing_details(listing):
    href_link = list(set(filter(None.__ne__, [tag.get('href') for tag in listing.findAll('a')])))
    link = 'http://www.wg-gesucht.de/en/' + href_link[0]
    cost = (listing.find('td', class_='ang_spalte_miete row_click').
                find('span').contents[1].contents[0].replace(' ',''))
    size = (listing.find('td', class_='ang_spalte_groesse row_click').
                find('span').contents[0].
                replace(' ','').replace('\n', ''))
    stadt = (listing.find('td', class_='ang_spalte_stadt row_click').
                find('span').contents[0].
                replace(' ','').replace('\n', ''))
    date_available = (listing.find('td', class_='ang_spalte_freiab row_click').
                    find('span').contents[0])
    
    return {'link': link,'cost': cost,'size': size,'stadt': stadt,'date_available': date_available}


def get_latest_listing_details():
    url = 'http://www.wg-gesucht.de/en/1-zimmer-wohnungen-in-Berlin.8.1.0.0.html?filter=fb0bdd36e1f2253e7bb343bba784e123c46b8103025e2e0a3a'
    soup = get_search_results(url)
    search_results = [get_listing_details(prop)
                      for prop in soup.findAll('tr', class_=re.compile('listenansicht0|listenansicht1'))]
    return pd.DataFrame(search_results).to_html()

In [9]:
search_results = get_latest_listing_details()

In [10]:
search_results.loc[0,'link']

'http://www.wg-gesucht.de/en/1-zimmer-wohnungen-in-Berlin-Kreuzberg.5607035.html'

In [15]:
print(search_results.to_html())

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>cost</th>
      <th>date_available</th>
      <th>link</th>
      <th>size</th>
      <th>stadt</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>235€</td>
      <td>12.06.16</td>
      <td>http://www.wg-gesucht.de/en/1-zimmer-wohnungen...</td>
      <td>28m²</td>
      <td>Kreuzberg</td>
    </tr>
    <tr>
      <th>1</th>
      <td>350€</td>
      <td>15.06.16</td>
      <td>http://www.wg-gesucht.de/en/1-zimmer-wohnungen...</td>
      <td>15m²</td>
      <td>Kreuzberg</td>
    </tr>
    <tr>
      <th>2</th>
      <td>479€</td>
      <td>15.07.16</td>
      <td>http://www.wg-gesucht.de/en/1-zimmer-wohnungen...</td>
      <td>37m²</td>
      <td>Moabit</td>
    </tr>
    <tr>
      <th>3</th>
      <td>420€</td>
      <td>15.06.16</td>
      <td>http://www.wg-gesucht.de/en/1-zimmer-wohnungen...</td>
      <td>35m²</td>
      <td>Wedding</td>
    </tr>
    


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html>
    <head>
    </head>
    <body>
    <table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>cost</th>
      <th>date_available</th>
      <th>link</th>
      <th>size</th>
      <th>stadt</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>235€</td>
      <td>12.06.16</td>
      <td>http://www.wg-gesucht.de/en/1-zimmer-wohnungen...</td>
      <td>28m²</td>
      <td>Kreuzberg</td>
    </tr>
    <tr>
      <th>1</th>
      <td>350€</td>
      <td>15.06.16</td>
      <td>http://www.wg-gesucht.de/en/1-zimmer-wohnungen...</td>
      <td>15m²</td>
      <td>Kreuzberg</td>
    </tr>
    <tr>
      <th>2</th>
      <td>479€</td>
      <td>15.07.16</td>
      <td>http://www.wg-gesucht.de/en/1-zimmer-wohnungen...</td>
      <td>37m²</td>
      <td>Moabit</td>
    </tr>
    <tr>
      <th>3</th>
      <td>420€</td>
      <td>15.06.16</td>
      <td>http://www.wg-gesucht.de/en/1-zimmer-wohnungen...</td>
      <td>35m²</td>
      <td>Wedding</td>
    </tr>
    <tr>
      <th>4</th>
      <td>334€</td>
      <td>01.07.16</td>
      <td>http://www.wg-gesucht.de/en/1-zimmer-wohnungen...</td>
      <td>25m²</td>
      <td>Lichtenberg</td>
    </tr>
    <tr>
      <th>5</th>
      <td>905€</td>
      <td>01.09.16</td>
      <td>http://www.wg-gesucht.de/en/1-zimmer-wohnungen...</td>
      <td>22m²</td>
      <td>Mitte</td>
    </tr>
    <tr>
      <th>6</th>
      <td>750€</td>
      <td>01.08.16</td>
      <td>http://www.wg-gesucht.de/en/1-zimmer-wohnungen...</td>
      <td>39m²</td>
      <td>Schleckheim</td>
    </tr>
    <tr>
      <th>7</th>
      <td>560€</td>
      <td>15.07.16</td>
      <td>http://www.wg-gesucht.de/en/1-zimmer-wohnungen...</td>
      <td>42m²</td>
      <td>Weißensee</td>
    </tr>
    <tr>
      <th>8</th>
      <td>599€</td>
      <td>15.06.16</td>
      <td>http://www.wg-gesucht.de/en/1-zimmer-wohnungen...</td>
      <td>41m²</td>
      <td>Schöneberg</td>
    </tr>
    <tr>
      <th>9</th>
      <td>510€</td>
      <td>01.08.16</td>
      <td>http://www.wg-gesucht.de/en/1-zimmer-wohnungen...</td>
      <td>39m²</td>
      <td>PrenzlauerBerg</td>
    </tr>
    <tr>
      <th>10</th>
      <td>350€</td>
      <td>02.07.16</td>
      <td>http://www.wg-gesucht.de/en/1-zimmer-wohnungen...</td>
      <td>31m²</td>
      <td>Friedrichshain</td>
    </tr>
    <tr>
      <th>11</th>
      <td>285€</td>
      <td>01.08.16</td>
      <td>http://www.wg-gesucht.de/en/1-zimmer-wohnungen...</td>
      <td>38m²</td>
      <td>Tiergarten</td>
    </tr>
    <tr>
      <th>12</th>
      <td>790€</td>
      <td>13.06.16</td>
      <td>http://www.wg-gesucht.de/en/1-zimmer-wohnungen...</td>
      <td>32m²</td>
      <td>Friedrichshain</td>
    </tr>
    <tr>
      <th>13</th>
      <td>492€</td>
      <td>04.07.16</td>
      <td>http://www.wg-gesucht.de/en/1-zimmer-wohnungen...</td>
      <td>42m²</td>
      <td>Kreuzberg</td>
    </tr>
    <tr>
      <th>14</th>
      <td>760€</td>
      <td>01.07.16</td>
      <td>http://www.wg-gesucht.de/en/1-zimmer-wohnungen...</td>
      <td>30m²</td>
      <td>Mitte</td>
    </tr>
    <tr>
      <th>15</th>
      <td>560€</td>
      <td>01.07.16</td>
      <td>http://www.wg-gesucht.de/en/1-zimmer-wohnungen...</td>
      <td>32m²</td>
      <td>Steglitz</td>
    </tr>
    <tr>
      <th>16</th>
      <td>795€</td>
      <td>15.07.16</td>
      <td>http://www.wg-gesucht.de/en/1-zimmer-wohnungen...</td>
      <td>53m²</td>
      <td>PrenzlauerBerg</td>
    </tr>
    <tr>
      <th>17</th>
      <td>550€</td>
      <td>01.07.16</td>
      <td>http://www.wg-gesucht.de/en/1-zimmer-wohnungen...</td>
      <td>38m²</td>
      <td>Friedrichshain</td>
    </tr>
    <tr>
      <th>18</th>
      <td>429€</td>
      <td>14.07.16</td>
      <td>http://www.wg-gesucht.de/en/1-zimmer-wohnungen...</td>
      <td>26m²</td>
      <td>Lichtenberg</td>
    </tr>
    <tr>
      <th>19</th>
      <td>380€</td>
      <td>15.07.16</td>
      <td>http://www.wg-gesucht.de/en/1-zimmer-wohnungen...</td>
      <td>15m²</td>
      <td>Friedenau</td>
    </tr>
  </tbody>
</table>
    </body>
 </html>